Data preparation process for Food Industry prediction modeling
==================
**Prepared by** : Grej - Jan. 29, 2019

###### Overview:
It is filtered to pre-registrants only with P18 codes.  
This means all uploaded files from the organizers were not included.


In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import geopy.distance
import math

The data is composed of 3 shows, Gulfood Manufacturing, AFS and Gulfood. The data needs to be loaded separately per show.

In [2]:
os.chdir(r'C:\Users\User\Documents\Data_Science_Projects\food-expo-attendee-prediction-project')

# encoding = latin-1 was used here due to the characters that are unreadable when using the standard utf-8
gfm = pd.read_csv(r'.\data\gfm.csv', encoding='latin-1')
afs = pd.read_csv(r'.\data\afs.csv', encoding='latin-1')
gfd = pd.read_csv(r'.\data\glf.csv', encoding='latin-1')

# add show columns
gfm['show'] = 'gfm'
afs['show'] = 'afs'
gfd['show'] = 'gfd'

##### 1. Create a function that merges the columns with codes in it and drops columns that are not useful to the modeling.
  
The columns which are coded needs to decoded.  
To do this, we need to first merge and process later.  
Some columns also need to be deleted as it has no use for this purpose.

In [3]:
## merge and drop columns
def pre_process(data, merge_columns, show_date): # merge_columns and drop_columns are relative to the show
    data['key_id'] = pd.to_numeric(data['key_id'])
    data['merged'] = data[merge_columns].apply(lambda x: ']' + x.astype(str), axis=1).apply(lambda x: r''.join(x.astype(str)), axis=1).str.replace(r' ', r'')
    data = data[['key_id', 'State', 'Country', 'Date Created', 'Email',
       'Website', 'show', 'merged']]
    data['show_date'] = pd.to_datetime(show_date)
    return data

In [4]:
# merge and drop for GFM
merge_columns = ['Company\'s Main Activity', 'Company Industry*',
       'Company Headquarter', 'Company\'s years in business*',
       'Company\'s size (No. of employees)*', 'Job Function*',
       'Job Title / Designation*', 'Job/Purchasing Role*',
       'My business goals for attending* (Select up to your top 3 reasons, in order of priority)',
       'I source at other events:*',
       'I follow Gulfood Manufacturing on social media*',
       'Content I am interested in* (Please select up to 3) - to be deleted',
       'Products I am interested in (select all that apply, minimum 1)',
       'Product Category: INGREDIENTS', 'Product Category: PROCESSING',
       'Product Category: PROCESSING - General Processing Technology',
       'Product Category: PROCESSING - Industry Focus - Bakery',
       'Product Category: PROCESSING - Industry Focus - Beverage',
       'Product Category: PROCESSING - Industry Focus - Meat / Poultry / Fish / Seafood',
       'Product Category: PROCESSING - Refrigeration Plants',
       'Product Category: PROCESSING - Food Safety & Quality',
       'Product Category: PACKAGING',
       'Product Category: PACKAGING - Packaging',
       'Product Category: PACKAGING - Filling & Sealing',
       'Product Category: PACKAGING - Inspecting & Checking',
       'Product Category: PACKAGING - Weighing Technology',
       'Product Category: PACKAGING - Denesting Machines',
       'Product Category: PACKAGING - Packaging Materials',
       'Product Category: PACKAGING - Printing & Labelling',
       'Product Category: SUPPLY CHAIN SOLUTIONS',
       'Product Category: SUPPLY CHAIN SOLUTIONS - Internal Transportation Equipment',
       'Product Category: SUPPLY CHAIN SOLUTIONS - External Transportation Equipment',
       'Product Category: SUPPLY CHAIN SOLUTIONS - Warehousing & Storage',
       'Product Category: SUPPLY CHAIN SOLUTIONS - Logistics',
       'Product Category: AUTOMATION & CONTROLS',
       'Product Category: MISCELLANEOUS', 'Overall Attendance']

gfm = pre_process(gfm, merge_columns=merge_columns, show_date='11/6/2018')

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [5]:
# merge and drop for AFS
merge_columns = ['CATEGORY', 'Show Selection', 'X Coded Show Selection', 'Gender',
       'My Job Function', 'My Job Role', 'My companys main activity is',
       'My Companys Line of Business / Industry', 'My company size is',
       'SEAFEX - I am interested in sourcing the following solutions/products ? (Select 3 Maximum)',
       'SPECIALTY - I am interested in sourcing the following solutions/products ? (Select 3 Maximum)',
       'YUMMEX - MAIN - I am interested in sourcing the following solutions/products ? (Select 3 Maximum)',
       'YUMMEX YU01 SUB Bagels - Bakery/Cakes/Dessert - I am interested in sourcing the following solutions/products ? (Select 3 Maximum)',
       'YUMMEX YU02 SUB Confectionery - I am interested in sourcing the following solutions/products ? (Select 3 Maximum)',
       'YUMMEX YU03 SUB Snacks - I am interested in sourcing the following solutions/products ? (Select 3 Maximum)',
       'YUMMEX YU04 SUB Chocolate / Chocolate  Products - I am interested in sourcing the following solutions/products ? (Select 3 Maximum)',
       'GULFHOST MAIN - I am interested in sourcing the following solutions/products ? (Select 3 Maximum)',
       'GULFHOST - GH01 SUB - I am interested in sourcing the following solutions/products ? (Select 3 Maximum)',
       'GULFHOST - GH02 SUB - I am interested in sourcing the following solutions/products ? (Select 3 Maximum)  ',
       'GULFHOST - GH03 SUB - I am interested in sourcing the following solutions/products ? (Select 3 Maximum) ',
       'GULFHOST - GH04 SUB - I am interested in sourcing the following solutions/products ? (Select 3 Maximum) ',
       'GULFHOST - GH05 SUB - I am interested in sourcing the following solutions/products ? (Select 3 Maximum) ',
       'Would you also be interested in visiting any of the below co-located shows?',
       'NATIONALITY - ONSITE', 'Promo Codes - GCL (Common for all 4 shows) ',
       'Promo Codes - SEAFEX', 'Promo Codes - SFF ',
       'Promo Codes - Gulf Host ', 'Promo Codes - YUMMEX ', 'Age',
       'JOB FUNCTION - DWTC - EXHIBITOR',
       'DESIGNATION LEVEL - DWTC - EXHIBITOR', 'EXHIBITOR SEGRAGATION - DWTC',
       'TERMS & CONDITIONS', 'UPLOAD CODES', 'REGISTRATION TYPE', 'STATIONS',
       'DTCM DEFAULT CODES', 'GATES', 'POST SHOW DTCM', 'OVERALL ATTENDANCE',
       'UTM Values', 'EMAIL STATUS (CAMPAIGN NAME)', 'ONSITE ATTENDANCE CODES',
       'OFFICE STATIONS', 'DATABASE MANAGEMENT', 'Language Pages',
       'Promo Code', 'DWTC - Country of Company ', 'DWTC - NATIONALITY',
       'DROP OFF MANAGEMENT', 'FREE OF CHARGE', 'Upload Codes ', 'EXHIBITOR',
       'Registrations Prepop', 'Email Broadcast', 'XCODES',
       'TELEMARKETING UPLOAD', 'Exhibitor Type', 'UPLOAD CODES ONSITE',
       '(Import): API Management', '*Onsite Midday Codes',
       '*Onsite Attendee Report Codes',
       '*Onsite Media Form: Please indicate the type of media you are from:',
       '*Onsite Fast Track Drop Codes', 'PRE-REG TYPE', 'Unknown Codes',
       '*Onsite - Conference Scanners Attendees', 'Long Data Entry Station']

afs = pre_process(afs, merge_columns=merge_columns, show_date='10-30-2019')

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [6]:
# merge and drop for GFD
merge_columns = ['Category',
       'Registration Type', 'Language Page', 'Package Code', 'My Job Function',
       'Job Role', 'Gender', 'My Company\'s Main Activity',
       'My Main Objective to Visiting the Show:',
       'I am interested in the following food/beverage products:',
       'I am interested in dry goods',
       'I am interested in BEVERAGES SOFT DRINKS',
       'I am interested in BEVERAGES  BEVERAGES HOT',
       'I am interested in DAIRY', 'I am interested in MEAT & POULTRY',
       'I am interested inCHILLED & FRESH FOOD',
       'I am interested in FROZEN FOOD', 'I am interested in SPECIALITY',
       'I am interested inMISCELLANEOUS',
       'I confirm that I have read and understood Gulfood 2018 Admission Policy. ',
       'Nationality - DWTC', 'PROMO CODE', 'Discount Codes', 'Priority Codes',
       'PAYMENT STATUS ', 'PAYMENT CHOICE ', 'PAYMENT MODE ',
       'PAYMENT DETAILS ', 'MEDIA - SOCIAL', 'MEDIA - MISC',
       'Designation Level - Exhibitor', 'Job Function - Exhibitor',
       'Age Group - Exhibitor', 'Exhibitor Segregation', 'Database Management',
       'Code Source (0)', 'DTCM Basket Codes', 'DTCM Basket Codes - Temporary',
       'DTCM Transaction Status', 'UTM Values', 'Email Broadcast', 'PPOP Data',
       'Station Names', 'Miscellaneous codes',
       'For International Visitors Only: (Onsite Form)',
       'I follow Gulfood on (Onsite Form)',
       'My Business Goals for Attending (Onsite Form)',
       'Do you source at other events? (Onsite Form)',
       'MEDIA ONSITE FORM: Please indicate the type of media you are from (Select all that apply)',
       'Nationality Onsite', 'Onsite File Upload Back Office',
       'Exhibitor Uploads Zahraa', 'Unknown Codes - Checked Onsite', 'SPLANB',
       'Wechat Registration', 'Onsite Report Codes', 'Onsite Attendance Codes',
       'Onsite Print Codes', 'X Code Payment Codes', 'DTCM / Attendance Codes',
       'Pre Reg File Uploads', 'Overall Attendance',
       'Overall Payment Status - Unique', 'Overall Conference Package 1',
       'Overall Conference Package 2']

gfd = pre_process(gfd, merge_columns=merge_columns, show_date='2-18-2018')

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


##### 2. Add show names to codes and append to make 1 dataset

In [7]:
def replace_codes(data, show):
    data['merged'] = r']'+data['merged']
    data['merged'] = data['merged'].str.replace(r']', r']' + show)
    return data
gfm = replace_codes(gfm, show='GFM')
afs = replace_codes(afs, show='AFS')
gfd = replace_codes(gfd, show='GFD')

gfd = gfm.append(afs, ignore_index=True).append(gfd, ignore_index=True)

Feature Engineering
======================

##### 3. Add email and website features
This part is a feature engineering process.  
The logic behind is that registrants that has entered websites and emails might have correlation to those who attend.  
One reason might be because those who have websites and emails are more interested and their company's are active in the industry.

In [8]:
def with_website(data):
    if 'Website' in data.columns.values:
        data.loc[data['Website']==" ", 'with_website'] = 0
        data.loc[data['Website']!=" ", 'with_website'] = 1
        return data
    else:
        print('There is no website column')

def with_email(data):
    if 'Email' in data.columns.values:
        data.loc[data['Email']==" ", 'with_email'] = 0
        data.loc[data['Email']!=" ", 'with_email'] = 1
        return data
    else:
        print('There is no Email column')

gfd = with_website(gfd)
#gfd = with_email(gfd)
gfd = gfd.drop(['Email', 'Website'], axis=1)

##### 4. Add days_to_go and weeks_to_go feature
The hypothesis is those who register close to the date of the show are more likely to attend.

In [9]:
def days_to_go_reg(data):
    if 'Date Created' in data.columns.values:
        data['Date Created'] = pd.to_datetime(data['Date Created'])
        difference = data['show_date'] - data['Date Created']
        return difference
    else:
        print('There is no Date Created column')

difference = days_to_go_reg(gfd)
gfd['days_to_go'] = difference.dt.days
gfd['weeks_to_go'] = round(gfd['days_to_go']/7)
gfd = gfd.drop(['Date Created', 'show_date'], axis=1)

##### 5. Cleanup the country and group into regions, add the distance of countries from UAE

In [10]:
# Load regions data
region = pd.read_excel(r'.\data\region.xlsx')

In [11]:
def cleanup_country(data, region):
    if 'Country' in data.columns.values:
        data.loc[pd.isnull(data['Country']), 'Country'] = 'United Arab Emirates' # replace blank countries with UAE
        data.loc[data['Country']=='', 'Country'] = 'United Arab Emirates' # replace blank countries with UAE
        data.loc[data['Country']==' ', 'Country'] = 'United Arab Emirates' # replace blank countries with UAE
        
        data = data.merge(region, left_on = 'Country', right_on = 'country', how = 'left')
        no_region = data.loc[pd.isnull(data['region_2'])]
        data = data.drop('Country', axis=1)
        return data, no_region
    else:
        print('There is no Country column')

gfd, no_region = cleanup_country(gfd, region)

The distance of the country from UAE might be also a factor.  
The hypothesis is those who come from farther places are less likely to attend.

In [12]:
# calculate distance of country from UAE.

def haversine(lon1, lat1, lon2, lat2):
    from math import radians, cos, sin, asin, sqrt
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

distance = pd.Series([])
for i in range(len(gfd.index)):
    lon1 = 53.847818
    lat1 = 23.424076
    lon2 = gfd.loc[i,['longitude']]
    lat2 = gfd.loc[i,['latitude']]
    dist = pd.Series(haversine(lon1, lat1, lon2, lat2))
    distance = distance.append(dist, ignore_index=True)
    
gfd['distance'] = distance

##### 6. Create groupings for UAE States

In [13]:
def create_state_group(data):
    if 'State' in data.columns.values:
        data['State'] = data['State'].str.lower().str.replace(r' ', r'_')
        data.loc[data['country']!= 'United Arab Emirates', 'State'] = 'international_state'
        data.loc[data['State']== r' ', 'State'] = 'dubai'
        data.loc[data['State']== r'_', 'State'] = 'dubai'
        data.loc[data['State']== r'', 'State'] = 'dubai'
        return data
data = create_state_group(gfd)


##### 7. Dummify all responses

In [14]:
# Load the codes data
codes = pd.read_excel(r'.\data\codes.xlsx')

In [15]:
codes[codes['show']=='AFS'][codes['included']=='Y']

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,show,question,code,decode,text_answer,included,job_rank
0,AFS,Overall Attendance,AFSATTENDED,attended,No,Y,NaN
1,AFS,CATEGORY,AFSVIS,visitor,No,Y,NaN
3,AFS,CATEGORY,AFSEXH,exhibitor,No,Y,NaN
4,AFS,CATEGORY,AFSMED,media,No,Y,NaN
5,AFS,CATEGORY,AFSORG,organiser,No,Y,NaN
6,AFS,CATEGORY,AFSVIP,vip,No,Y,NaN
7,AFS,CATEGORY,AFSHB,hosted_buyer,No,Y,NaN
8,AFS,CATEGORY,AFSDEL,delegate,No,Y,NaN
9,AFS,CATEGORY,AFSSPK,speaker,No,Y,NaN
10,AFS,CATEGORY,AFSSTU,student,No,Y,NaN


Choose what show data to use
================

In [16]:
data = data[data['show']=='gfd'] ## ----------------> use gulfood data only

In [17]:
def dummify_responses(data, codes):
    columns_to_check = {'merged', 'key_id'}
    for cols in columns_to_check:
        if cols not in data.columns.values:
            print('There is no ',cols,' column')
            break
    else:
        responses = data['merged'].str.split(r']', expand=True)
        responses['key_id'] = data['key_id']
        responses = responses.melt(id_vars=['key_id'], value_name = 'code')
        responses['value'] = 1
        
        responses = responses.merge(codes, left_on = 'code', right_on = 'code', how = 'left')
        responses = responses.loc[responses['included'] == 'Y']
        responses = responses.drop(['show', 'question', 'code', 'text_answer', 'included', 'job_rank'], axis=1)
        responses = responses.pivot_table(index = ['key_id'], columns = 'decode', values = 'value', aggfunc = 'max')
        #responses.loc[responses['Attended']!=1, 'Attended'] = 0
        return responses

responses = dummify_responses(data, codes)
data = data.merge(responses, on='key_id', how = 'left').drop('merged', axis=1)

##### 8. Dummify all other categorical variables not included in responses

In [18]:
def dummify_columns(data, columns):
    columns_to_check = {'key_id'}
    for cols in columns_to_check:
        if cols not in data.columns.values:
            print('There is no ',cols,' column')
            break
    data_1 = data[columns]
    data_1 = pd.get_dummies(data_1[columns])
    data_1['key_id'] = data['key_id']
    data = data.merge(data_1, on='key_id', how = 'left')
    data = data.drop(columns, axis=1)
    return data

#data = data.drop('country', axis=True)
columns = ['State', 'country', 'region_1', 'region_2']
data = dummify_columns(data, columns)
data = data.fillna(0)

##### 9. Add company_reg_counts

In [19]:
# Load the codes data and merge with the data
company_reg_count = pd.read_pickle(r'.\data\output\company_reg_count.pkl')
data = data.merge(company_reg_count, on='key_id', how='left')

##### 10. Resample data to balance the number of observations per group of Attendance

In [20]:
'''# undersample Attendees to balance the data - to reduce bias, do it by show
def undersample_data(data):
    shows = ['gfm', 'afs', 'gfd']
    data_1 = pd.DataFrame()
    for show in shows:
        filter1 = (data['attended']==1) & (data['show']==show)
        filter2 = (data['attended']==0) & (data['show']==show)
        k = len(data[filter2].index)
        data_2 = data[filter1].sample(n=k, replace=True, random_state=1)
        data_3 = data[filter2]
        data_1 = data_1.append(data_2, ignore_index=True).append(data_3, ignore_index=True)
        data_1 = data_1.drop('show', axis=True)
    return data_1


data = balance_data(data)'''

"# undersample Attendees to balance the data - to reduce bias, do it by show\ndef undersample_data(data):\n    shows = ['gfm', 'afs', 'gfd']\n    data_1 = pd.DataFrame()\n    for show in shows:\n        filter1 = (data['attended']==1) & (data['show']==show)\n        filter2 = (data['attended']==0) & (data['show']==show)\n        k = len(data[filter2].index)\n        data_2 = data[filter1].sample(n=k, replace=True, random_state=1)\n        data_3 = data[filter2]\n        data_1 = data_1.append(data_2, ignore_index=True).append(data_3, ignore_index=True)\n        data_1 = data_1.drop('show', axis=True)\n    return data_1\n\n\ndata = balance_data(data)"

In [21]:
'''# oversample Attendees to balance the data - to reduce bias, do it by show
def oversample_data(data):
    smote = SMOTE(random_state=23)
    ids = data[['key_id', 'show']]
    y = data['attended']
    X = data.drop(['attended', 'key_id', 'show'], axis=1)
    data, attended = smote.fit_resample(X,y)
    data['attended'] = pd.Series(attended)
    data['key_id'] = ids['key_id']
    return data

data = oversample_data(data)
'''

"# oversample Attendees to balance the data - to reduce bias, do it by show\ndef oversample_data(data):\n    smote = SMOTE(random_state=23)\n    ids = data[['key_id', 'show']]\n    y = data['attended']\n    X = data.drop(['attended', 'key_id', 'show'], axis=1)\n    data, attended = smote.fit_resample(X,y)\n    data['attended'] = pd.Series(attended)\n    data['key_id'] = ids['key_id']\n    return data\n\ndata = oversample_data(data)\n"

##### 8. Save data as pickle

In [22]:
data.to_pickle(r'.\data\output\cleanData.pkl')

In [23]:
columns = pd.DataFrame({'cols' : data.columns})
columns = columns[1:]
columns = columns[columns['cols']!='attended']
columns.to_pickle(r'.\data\output\columns_used_for_model.pkl')

In [24]:
data.columns

Index(['key_id', 'show', 'with_website', 'days_to_go',
       'weeks_to_go', 'latitude', 'longitude', 'distance', 'arabic_page',
       'armed_forces_police',
       ...
       'region_1_Western Europe', 'region_2_Africa', 'region_2_Americas',
       'region_2_Australia-Asia', 'region_2_Europe', 'region_2_ME GCC',
       'region_2_ME Non-GCC', 'count_per_company', 'count_per_comp_website',
       'count_per_website'],
      dtype='object', length=279)

In [25]:
data['female']

0        0.0
1        0.0
2        0.0
3        0.0
4        1.0
5        0.0
6        0.0
7        0.0
8        0.0
9        0.0
10       0.0
11       0.0
12       0.0
13       0.0
14       0.0
15       0.0
16       1.0
17       0.0
18       0.0
19       0.0
20       1.0
21       0.0
22       0.0
23       0.0
24       1.0
25       1.0
26       1.0
27       1.0
28       0.0
29       0.0
        ... 
69085    0.0
69086    0.0
69087    0.0
69088    0.0
69089    0.0
69090    0.0
69091    0.0
69092    0.0
69093    0.0
69094    1.0
69095    1.0
69096    0.0
69097    1.0
69098    0.0
69099    0.0
69100    1.0
69101    0.0
69102    0.0
69103    0.0
69104    0.0
69105    1.0
69106    0.0
69107    0.0
69108    0.0
69109    1.0
69110    1.0
69111    0.0
69112    0.0
69113    0.0
69114    0.0
Name: female, Length: 69115, dtype: float64